In [1]:
# import packages for cleaning
import pandas as pd

In [2]:
# create dataframes from sourcefiles

df2010 = pd.read_csv('/Users/Elliot/Desktop/SocrataInput/VA Hospital compare data - export - 2010_Q4.csv')
df2011 = pd.read_csv('/Users/Elliot/Desktop/SocrataInput/VA Hospital compare data - export - 2011_Q4.csv')
df2012 = pd.read_csv('/Users/Elliot/Desktop/SocrataInput/VA Hospital compare data - export - 2012_Q4.csv')
df2013 = pd.read_csv('/Users/Elliot/Desktop/SocrataInput/VA Hospital compare data - export - 2013_Q4.csv')
df2014 = pd.read_csv('/Users/Elliot/Desktop/SocrataInput/VA Hospital compare data - export - 2014_Q3.csv')

In [3]:
# create dataframe from VA json file

va_json = pd.read_json('https://raw.githubusercontent.com/department-of-veterans-affairs/VHA-Facilities/master/VAFacilityLocation.json', orient='columns')

In [4]:
# drop irrelevant columns

va_json = va_json.drop(va_json.columns[[0,1,2,3]], axis = 1)

In [5]:
# orient dataframe based off column id "VA Facility Data" and read contents

df_va_json = pd.read_json((va_json.VAFacilityData).to_json(), orient='index')

In [6]:
# keep health centers only

df_va_json = df_va_json[(df_va_json.type == 'VA Medical Center') | (df_va_json.type == 'Community Based Outpatient Clinic') |
                        (df_va_json.type == 'VA Health Care System') | (df_va_json.type == 'Outpatient Clinic')]

In [7]:
# drop irrelevant columns

df_va_json = df_va_json.drop(df_va_json.columns[[0,2,4,7,8,9,10,12,13]], axis = 1)

In [8]:
# add in missing data from facility id '612A4'

df_va_missing = pd.DataFrame({'city': ['Mather'], 'facility_id': ['612A4'], 'latitude': [38.57306], 'longitude': [-121.295515], 'state': ['CA']})

In [9]:
# Concatenate dataframes

frames = [df_va_json, df_va_missing]
df_va_json = pd.concat(frames)

In [10]:
# rename column names

df_va_json = df_va_json.rename(columns={'city': 'CITY', 'facility_id': 'FACILITY', 'latitude': 'LATITUDE', 'longitude':'LONGITUDE', 'state': 'STATE'})

In [11]:
# dtop duplicates

df_va_json = df_va_json.drop_duplicates('FACILITY')

In [12]:
# verify count

df_va_json.count()

CITY         892
FACILITY     892
LATITUDE     890
LONGITUDE    890
STATE        892
dtype: int64

In [13]:
# Assign new column based on file name year

df2010 = df2010.assign(YEAR = 2010)
df2011 = df2011.assign(YEAR = 2011)
df2012 = df2012.assign(YEAR = 2012)
df2013 = df2013.assign(YEAR = 2013)
df2014 = df2014.assign(YEAR = 2014)

In [14]:
# assign new column based on file name quarter

df2010 = df2010.assign(QUARTER = 'Q4')
df2011 = df2011.assign(QUARTER = 'Q4')
df2012 = df2012.assign(QUARTER = 'Q4')
df2013 = df2013.assign(QUARTER = 'Q4')
df2014 = df2014.assign(QUARTER = 'Q3')

In [15]:
# concatenate into single dataframe

frames = [df2010,df2011,df2012,df2013,df2014]

df = pd.concat(frames)

In [16]:
# create new dataframe for VISN average values (where FACILITY == 0)

df_avg = df[df.FACILITY == '0']

# write to csv into directory

df_avg.to_csv('VA Aspire Dashboard Average.csv')

In [17]:
# drop rows with VISN average values based on FACILITY value

df = df[df.FACILITY != '0']

In [18]:
# display df count

df.count()

MEASURE     62160
TYPE        62160
HILO        33300
DECS        62160
GOAL        25456
AVG         35964
VISN        62160
FACILITY    62160
LOCATION    62160
VALUE       25138
SCORE       62160
TREND       13761
NOTES       62160
YEAR        62160
QUARTER     62160
dtype: int64

In [19]:
# Merging the df dataframe with the df_va_json dataframe based off values in the df.FACILITY column
# and matching them with values in the df_va_json.FACILITY column

df = pd.merge(left=df, right=df_va_json, how='left', left_on='FACILITY', right_on='FACILITY')

In [20]:
# Verifying counts are correct after the left merge

df.count()

MEASURE      62160
TYPE         62160
HILO         33300
DECS         62160
GOAL         25456
AVG          35964
VISN         62160
FACILITY     62160
LOCATION     62160
VALUE        25138
SCORE        62160
TREND        13761
NOTES        62160
YEAR         62160
QUARTER      62160
CITY         62160
LATITUDE     62160
LONGITUDE    62160
STATE        62160
dtype: int64

In [21]:
# Dropping column "LOCATION" as this data has been separated out into other columns

df = df.drop('LOCATION', 1)

In [22]:
# Creating columns for region based off of this VA Aspire map: http://www.benefits.va.gov/reports/aspiremap.asp/

region = []

for row in df.STATE:
    if (row == 'AR' or row == 'CO' or row == 'LA' or row == 'MS' or row == 'MT' or
        row == 'OK' or row == 'TX' or row == 'UT' or row == 'WY'):
        region.append('Continental')
    elif (row == 'IL' or row == 'IN' or row == 'IA' or row == 'KS' or row == 'MI' or
          row == 'MN' or row == 'MO' or row == 'NE' or row == 'ND' or row == 'OH' or
          row == 'SD' or row == 'WI'):
        region.append('Midwest')
    elif (row == 'CT' or row == 'DE' or row == 'ME' or row == 'MD' or row == 'MA' or
          row == 'NH' or row == 'NJ' or row == 'NY' or row == 'NC' or row == 'PA' or
          row == 'RI' or row == 'VT' or row == 'VA' or row == 'WV'):
        region.append('North Atlantic')
    elif (row == 'AK' or row == 'AZ' or row == 'CA' or row == 'HI' or row == 'ID' or
          row == 'NV' or row == 'NM' or row == 'OR' or row == 'WA'):
        region.append('Pacific')
    elif (row == 'AL' or row == 'FL' or row == 'GA' or row == 'KY' or row == 'SC' or 
          row =='TN'):
        region.append('Southeast')
    else:
        region.append('Other')

df['REGION ']= region

In [23]:
# Create new measure dataframe and split column: MEASURE based on first '-'

df_measure = df['MEASURE'].str.split('-', expand=True, n=1)

In [24]:
# rename column headers for df_measure

df_measure = df_measure.rename(columns={0: 'CATEGORY', 1: 'MEASURE'})

In [25]:
# verify df_measure before concatenation

df_measure.head()

,CATEGORY,MEASURE
0,Safety,Healthcare associated infections - MRSA infec...
1,Safety,Healthcare associated infections - MRSA infec...
2,Safety,Healthcare associated infections - MRSA infec...
3,Safety,Healthcare associated infections - MRSA infec...
4,Safety,Healthcare associated infections - MRSA infec...


In [26]:
# concatenate df_measure with df

df = pd.concat([df_measure, df], axis = 1)

In [27]:
# Verifying that the file looks good to publish 

df.head()

,CATEGORY,MEASURE,MEASURE,TYPE,HILO,DECS,GOAL,AVG,VISN,FACILITY,...,SCORE,TREND,NOTES,YEAR,QUARTER,CITY,LATITUDE,LONGITUDE,STATE,REGION
0,Safety,Healthcare associated infections - MRSA infec...,Safety - Healthcare associated infections - MR...,outcomes,lower,2,0,0.1992,1,402,...,4,-1,value within 10% of aspirational goal;,2010,Q4,Augusta,44.279805,-69.701815,ME,North Atlantic
1,Safety,Healthcare associated infections - MRSA infec...,Safety - Healthcare associated infections - MR...,outcomes,lower,2,0,0.1992,1,405,...,4,0,value within 10% of aspirational goal;,2010,Q4,White River Junction,43.648498,-72.342911,VT,North Atlantic
2,Safety,Healthcare associated infections - MRSA infec...,Safety - Healthcare associated infections - MR...,outcomes,lower,2,0,0.1992,1,518,...,0,NaN,no data available;,2010,Q4,Bedford,42.502902,-71.272671,MA,North Atlantic
3,Safety,Healthcare associated infections - MRSA infec...,Safety - Healthcare associated infections - MR...,outcomes,lower,2,0,0.1992,1,523A4,...,4,0,value within 10% of aspirational goal;,2010,Q4,West Roxbury,42.270702,-71.171977,MA,North Atlantic
4,Safety,Healthcare associated infections - MRSA infec...,Safety - Healthcare associated infections - MR...,outcomes,lower,2,0,0.1992,1,523A5,...,0,NaN,no data available;,2010,Q4,Brockton,42.065109,-71.054714,MA,North Atlantic


In [28]:
#Looks good! Ready to publish.

df.to_csv('VA Aspire Dashboard.csv', index=False)

In [ ]:
###WILL NOT RUN WITHOUT MODIFICATION###
# Will not work without updating credential values, dataset id values
# Also should be run from the command prompt/terminal

# Source for test upload of data to existing dataset, read more here: https://github.com/xmun0x/sodapy
# must install package: pip install sodapy

from sodapy import Socrata

client = Socrata("url.socrata.com", "MyAppToken", username = "myemail@mydomain.com", password = "MyP@s$W0RD!")

#Dataset has to be created on socrata.com prior to running this field in order to generate Socrata '4x4' id

data = open("VA Aspire Dashboard.csv")

client.replace("####-####", data)

#close connection

client.close()